In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
from langchain.llms import ctransformers

c:\Users\Lenovo\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY = "2bd5fd9f-2c56-42f9-aa1b-037960262fff"
PINECONE_API_ENV = "gcp-starter"

In [4]:
def load_data(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [6]:
extracted_data = load_data('../data/')

In [7]:
extracted_data[:5]

[Document(page_content='', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 0}),
 Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 1}),
 Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B1', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 2}),
 Document(page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow, Manager, Imaging and Multimedia\nContent\nRobyn V . Young, Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and\nMultimedia Con

In [8]:
#create chunks for document
def text_split(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size= 100, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(data)
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("Length of chunks: ",len(text_chunks))

Length of chunks:  37348


In [10]:
text_chunks[:4]

[Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 1}),
 Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 2}),
 Document(page_content='DEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B1', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 2}),
 Document(page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 3})]

In [11]:
def download_hugging_face_embedding():
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

In [13]:
embeddings = download_hugging_face_embedding()

In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [15]:
query_result = embeddings.embed_query("hellow world")
print(len(query_result))

384


In [16]:
query_result

[-0.03862404078245163,
 0.023439398035407066,
 0.002642315346747637,
 0.012942559085786343,
 -0.010668904520571232,
 -0.1494266539812088,
 0.07319582253694534,
 -0.025585532188415527,
 -0.054648496210575104,
 0.021850230172276497,
 0.06717637181282043,
 0.008475841023027897,
 0.036259159445762634,
 0.0003588153049349785,
 -2.1329669834813103e-05,
 -0.023354068398475647,
 0.0016247483436018229,
 -0.017904071137309074,
 -0.18061451613903046,
 -0.029042942449450493,
 0.029159652069211006,
 0.040664348751306534,
 -0.013561608269810677,
 0.03132638335227966,
 -0.07914232462644577,
 -0.03398185595870018,
 0.038639798760414124,
 0.08922843635082245,
 0.030200937762856483,
 -0.03327992558479309,
 0.017174456268548965,
 0.03981738165020943,
 0.12325078994035721,
 -0.014756337739527225,
 -0.02620898187160492,
 0.04922116920351982,
 -0.03202835097908974,
 -0.0857212096452713,
 -0.023259660229086876,
 0.049486249685287476,
 0.05224711075425148,
 -0.08279266953468323,
 0.0016369631048291922,
 -0.02